# L13b: Drug Combination Design using Reinforcement Learning
In this lab, we will explore the use of reinforcement learning (RL) techniques to design effective drug combinations. The goal is to identify combinations of drugs that can work synergistically to treat diseases more effectively than individual drugs alone.

> __Learning Objectives:__
> 
> By the end of this lab, you will be able to:
> Three learning objectives go here.

Let's get started!
___

## Problem
The Drug Cocktail Design Problem involves selecting a combination of drugs to create a cocktail that maximizes therapeutic effectiveness while staying within budget and safety constraints. Each drug has attributes such as dosage, mechanism of action, and side effects, which contribute to the overall effectiveness of the cocktail. The drug cocktail design problem can be formulated as the following optimization problem:
$$
\boxed{
\begin{align*}
\text{maximize} \quad & U(n_1, \dots, n_K) = \kappa(\gamma) \prod_{i=1}^{K} n_i^{\gamma_i} \\
\text{subject to} \quad & \sum_{i=1}^{K} c_i (n_i\;W) \leq B\quad\text{(budget constraint)}\\
& \sum_{i=1}^{K} n_i \leq S\quad\text{(safety constraint)}\\
& n_i^{\text{min}} \leq n_i \leq n_i^{\text{max}} \quad \forall i \in \{1, \dots, K\}
\end{align*}}
$$
where the objective function is a Cobb-Douglas utility function representing the effectiveness of the drug cocktail, and the constraints ensure that the cocktail stays within budget and safety limits. The design variables are the concentrations of each drug in the cocktail, denoted by $n_i$ for drug $i$ (units: mg/kg), $K$ is the total number of available drugs, and $W$ is the weight of the patient (units: kg). The coefficients $\gamma_i$ represent the effectiveness of each drug type, which can be learned from historical data or set based on expert knowledge.

The first set of constraints ensures that the total cost of the drug cocktail does not exceed a specified budget $B$, where $c_i$ is the cost per unit concentration (e.g., USD/mg) of drug $i$. The second set of constraints ensures that the total concentration of all drugs in the cocktail does not exceed a safety limit $S$. Finally, each drug concentration is bounded by its minimum and maximum allowable levels, denoted by $n_i^{\text{min}}$ and $n_i^{\text{max}}$, respectively.
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include-solution.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Implementation
Let's implement the `experiment(...)` function that is used by our reinforcement learning agent to evaluate drug cocktail designs. This function will compute the utility of a given drug cocktail based on the Cobb-Douglas utility function and apply the necessary constraints.

> __What is going on in the `experiment(...)` function?__
> 
> This function takes in a context object, which contains information about the drug types, their costs, and levels. It also takes in two integers, `s` and `a`, which represent the state and action, respectively. The function computes the utility of the drug cocktail based on the Cobb-Douglas utility function and checks if the cocktail meets the budget and safety constraints. If the constraints are met, it returns the computed utility; otherwise, it returns a penalty value.

Implement the `experiment(...)` function in the code cell below:

In [2]:
function experiment(context::MyExperimentalDrugCocktailContext, s::Int64, a::Int64)
    
    # initialize -
    K = context.K; # number of drug types
    m = context.m; # number of drug levels per type {high, nominal, low}
    levels = context.levels; # drug levels
    costs = context.cost; # drug costs
    W = context.W; # drug interaction matrix
    γ = context.γ; # discount factor
    B = context.B; # budget in USD
    N = 2^K; # number of actions
    M = m^K; # number of states
    constraint_penalty = -10000.0; # penalty for violating budget constraint

    # compute the vector representations for state and action -
    sᵢ = digits(s, base=m, pad=K); # state vector representation
    aᵢ = digits(a, base=2, pad=K); # action vector representation
    if (a == N)
        aᵢ = digits(a-1, base=2, pad=K); # adjust for all-drug case
    end

    # which drugs are being administered? -
    S₊ = findall(x -> x == 1, aᵢ); # indices of drugs being administered
    n = zeros(Float64, K); # initialize concentration vector 
    tmp = 1.0; # initialize utility value -
    for i ∈ eachindex(S₊)
        drug_index = S₊[i];
        level_index = sᵢ[drug_index];

        # compute the concentration -
        nᵢ = 0.0;
        if (level_index == 0) # high
            nᵢ = levels[drug_index].high;
        elseif (level_index == 1) # nominal
            nᵢ = levels[drug_index].nominal;
        elseif (level_index == 2) # low
            nᵢ = levels[drug_index].low;
        end
        n[drug_index] = nᵢ; # store concentration (we'll use this for later)

        # look up the efficacy -
        γᵢ = γ[drug_index]; # efficacy of drug i

        # update the utility value -
        tmp *= nᵢ^γᵢ;
    end

    # compute the scaling factor -
    κ = 1.0; # scaling factor for utility
    negative_gamma_flag = any(γ[S₊] .< 0);
    if (negative_gamma_flag == true)
        κ = -1.0; # flip sign if any gamma is negative
    end

    # compute the budget constraint -
    spent = 0.0; # total spent
    for i ∈ eachindex(S₊)
        drug_index = S₊[i];
        cᵢ = costs[drug_index]; # cost of drug i
        spent += cᵢ * n[drug_index]*W; # accumulate total spent
    end
    budget_violation = max(0.0, spent - B); # budget violation

    # choose a new state to try -
    s′ = rand(1:M); # new state at random

    # compute the reward (utility + constraint penalty) -
    U = κ * tmp + constraint_penalty * budget_violation; 

    (s′, U) # return new state and reward
end

experiment (generic function with 1 method)

### Constants
In this section, let's define some constants that will be used in our drug cocktail design problem. These constants include the number of drug types, their costs, and the levels of drug concentrations. See the comment next to each constant for its units, permitted values, and description.

In [3]:
K = 3; # number of drug types
m = 3;  # number of drug levels per type {high, nominal, low}
𝒜 = range(1, stop=2^K, step=1) |> collect; # TODO: What does this mean? (why is it 2^K?)
𝒮 = range(1, stop=m^K, step=1) |> collect; # TODO: What does this mean? (why is it m^K?)
W = 80.0; # weight of the patient in kg
B = 10000.0; # budget in USD

___

## Task 1: Setup the context model
In this task, we will set up the context model for our drug cocktail design problem. This involves defining a mutable struct that holds the necessary information about the drugs, their costs, and levels. 

In [4]:
contextmodel = let 

    # initialize - 
    costs = Dict{Int64, Float64}(); # cost per unit concentration (e.g., USD/mg) of drug i
    levels = Dict{Int64, NamedTuple}(); # levels of drug concentrations for drug i
    γ = Array{Float64}(undef, K); # effectiveness coefficients for each drug type

    # generate random cost data -
    for i ∈ 1:K
        costs[i] = rand(1.0:0.1:10.0); # random cost between 1.0 and 10.0 USD/mg
    end

    # generate random level data -
    for i ∈ 1:K
        high = rand(50.0:1.0:100.0);    # high concentration level in mg/kg
        nominal = rand(20.0:1.0:49.0);  # nominal concentration level in mg/kg
        low = rand(1.0:1.0:19.0);      # low concentration level in mg/kg
        levels[i] = (high=high, nominal=nominal, low=low);
    end

    # generate effectiveness coefficients -
    for i ∈ 1:K
        γ[i] = randn(); # random normal value
    end

    # build the model -
    model = build(MyExperimentalDrugCocktailContext, (

        K = K, # number of drug types
        m = m, # number of drug levels per type
        γ = γ, # effectiveness coefficients for each drug type (we don't know these a priori)
        B = B, # total budget in USD
        cost = costs, # cost per unit concentration (e.g., USD/mg) of drug i
        levels = levels, # levels of drug concentrations for drug i
        W = W # weight of the patient in kg
    ));
    
    model; # return the model
end;

## Task 2: Construct the Learning Agent Model
In this task, we will construct the learning agent model that will be used to evaluate different drug cocktail designs. This agent will run experiments using the `experiment(...)` function we implemented earlier and will learn to optimize the drug cocktail design over time such that the utility is maximized while adhering to the constraints.

In [5]:
mylearningagent = let

    # initialize -
    γ = 0.95; # discount factor (for future rewards)
    α = 0.1;  # learning rate
    Q = Array{Float64}(undef, length(𝒮), length(𝒜)); # Q-value table

    # fill the Q-table with zeros -
    fill!(Q, 0.0); # fast way to fill an array with a value

    # build the learning agent model -
    model = build(MyQLearningAgentModel, (
        γ = γ, # discount factor
        α = α, # learning rate
        Q = Q, # Q-value table
        states = 𝒮, # state space
        actions = 𝒜 # action space
    ));

    model; # return the model
end

MyQLearningAgentModel([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  18, 19, 20, 21, 22, 23, 24, 25, 26, 27], [1, 2, 3, 4, 5, 6, 7, 8], 0.95, 0.1, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

## Task 3: Let's let the agent learn!
In this task, we will allow the learning agent to interact with the environment and learn from its experiences. The agent will use the `experiment(...)` function to evaluate different drug cocktail designs and update its policy based on the observed rewards. 

In [ ]:
result = solve(mylearningagent, contextmodel; 
    maxsteps = 1000, δ = 0.0001, worldmodel = experiment);

In [7]:
result.Q

27×8 Matrix{Float64}:
 -2.54953e7   -1.01567e7  -5.0074e7   …  -5.9656e7   -6.36944e7  -7.78748e7
 -0.00723689   0.0        -2.91285e7     -1.10715e8  -1.63879e8   0.0
 -1.11068e8   -0.0446698  -9.98795e7      0.0        -9.78655e7  -1.65418e8
 -8.86294e6   -0.0616152  -3.5762e7       0.0        -6.63332e7  -1.23886e8
 -0.0151483   -0.0647906  -3.72483e6     -7.92813e7   0.0        -4.6808e7
 -3.5628e7    -0.151794   -1.4419e8   …  -1.00981e8   0.0        -8.44838e7
 -1.4791e7    -0.199582   -1.93792e7      0.0        -5.9216e7   -1.54817e8
 -0.0206731   -0.0781259  -0.011581      -3.32254e7  -1.026e8    -8.20124e7
 -4.356e7     -2.77592e7  -6.34032e7     -5.57096e7   0.0        -7.391e7
 -1.62357e7   -4.92974e7  -5.61685e7     -5.63255e7  -7.99613e7   0.0
  ⋮                                   ⋱   ⋮                      
 -9.50893e6   -1.02593e7  -2.32699e7     -1.54618e7   0.0        -1.13048e8
 -0.0237677   -1.00552e7  -4.34631e7     -4.12153e7  -4.13874e7  -2.14397e7
 -3.74641e7   -

Let's extract the policy from our trained agent.

In [10]:
π(s) = mypolicy(result.Q)[s]; # wow! that's fancy, what are we doing here??

In [12]:
π(1)

5

What's in the policy? Let's pick the best action for each state according to the learned Q-values. We'll display these in a table.

In [17]:
let

    # initialize -
    df = DataFrame();
    s = 1; # what state do we want to look at?
    a = π(s); # this gives me the best action *index* for state s
    N = 2^K; # number of actions
    M = m^K; # number of states
    γ = contextmodel.γ; # effectiveness coefficients for each drug type

    aᵢ = digits(a, base=2, pad=K); # action vector representation
    if (a == N)
        aᵢ = digits(a-1, base=2, pad=K); # adjust for all-drug case
    end
    S₊ = findall(x -> x == 1, aᵢ); # indices of drugs being administered

    for i ∈ eachindex(S₊)
        drug_index = S₊[i];
        level_index = digits(s, base=m, pad=K)[drug_index];

        level_str = "";
        if (level_index == 0)
            level_str = "high";
        elseif (level_index == 1)
            level_str = "nominal";
        elseif (level_index == 2)
            level_str = "low";
        end

        push!(df, (
            state = s,
            action = a,
            drug = drug_index,
            γ = γ[drug_index],
            level = level_str,
            q_value = result.Q[s, a]
        ));
    end

    display(df);

end

Row,state,action,drug,γ,level,q_value
,Int64,Int64,Int64,Float64,String,Float64
1,1,5,1,-1.04442,nominal,0.0
2,1,5,3,0.521895,high,0.0


## Summary
One concise and direct summary sentence goes here.

> __Key Takeaways:__
> 
> Three key takeaways go here.

One final concluding sentence goes here.
___